#### .pdf内のテキストを抽出  .csvファイル出力

In [ ]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import (
    LAParams,
    LTContainer,
    LTTextLine,
)

import glob
import re
import os
import pandas as pd

pdf_files= []

target_path = r'C:\Users\you-m\jupyter\*.pdf'

# target_pathで指定したフォルダ内のファイルのパスを格納
files = glob.glob(target_path, recursive=True)

# このうち「.pdf」がつくものだけを抽出し、pdf_filesに格納
for file in files:
    if '.pdf' in file:
        pdf_files.append(file)

# 「pdf_files」のリストに格納されたパスを表示
for pdf_file in pdf_files:
    print(pdf_file)

# 「pdf_files」のリストに格納されたパスの数を表示
len(pdf_files)

ix = 1
for pdf_file in pdf_files:
   
    # 抽出したテキストデータを格納する空リスト
    extracted_texts = []
    
    fp = open(pdf_file, 'rb')
    parser = PDFParser(fp)
    document = PDFDocument(parser)
    
    if not document.is_extractable:
        raise #PDFTextExtractionNotAllowed
    rsrcmgr = PDFResourceManager()
    laparams = LAParams()
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)

    interpreter = PDFPageInterpreter(rsrcmgr, device)
    #pages = list(document.get_pages())
    
    for page in PDFPage.create_pages(document):
        interpreter.process_page(page)
        layout = device.get_result()
        for l in layout:
            if isinstance(l, LTTextBoxHorizontal):
                text = l.get_text()
                extracted_texts.append(text)
    
    
    print(extracted_texts)
    df = pd.DataFrame(data = extracted_texts)
    df.to_csv("pdf_test" + str(ix) + ".csv",encoding='utf_8_sig')
    ix = ix + 1
